In [1]:
import pandas as pd 
import numpy as np 
import os, re, pathlib
from math import sqrt

## 1. Find the email activity rank for each user. 
- Email activity rank is defined by the total number of emails sent. 
- The user with the highest number of emails sent will have a rank of 1, and so on. Output the user, total emails, and their activity rank. 

- Order records by the total emails in descending order. Sort users with the same number of emails in alphabetical order.
- In your rankings, return a unique value (i.e., a unique rank) even if multiple users have the same number of emails. 
- For tie breaker use alphabetical order of the user usernames.

In [3]:
google_gmail_emails = pd.read_excel(f"{os.getcwd()}/data/Activity Rank.xlsx")
google_gmail_emails.head(3)

,id,from_user,to_user,day
0,0,6edf0be4b2267df1fa,75d295377a46f83236,10
1,1,6edf0be4b2267df1fa,32ded68d89443e808,6
2,2,6edf0be4b2267df1fa,55e60cfcc9dc49c17e,10


In [28]:
# tst=google_gmail_emails.groupby('from_user').agg({'to_user':'count'}).reset_index().rename(columns={'to_user':'total_email_sent'}).sort_values('total_email_sent', ascending=False)
# tst['rank']=tst['total_email_sent'].rank(method='first',ascending=False)

In [4]:
rank_rlt = google_gmail_emails.groupby('from_user')['to_user'].count().reset_index().rename(columns={'to_user':'total_emails'})
rank_rlt['rank'] = rank_rlt['total_emails'].rank(method='first', ascending=False)
rank_rlt = rank_rlt.sort_values(by='rank')
rank_rlt

,from_user,total_emails,rank
3,32ded68d89443e808,19,1.0
24,ef5fe98c6b9f313075,19,2.0
7,5b8754928306a18b68,18,3.0
6,55e60cfcc9dc49c17e,16,4.0
16,91f59516cb9dee1e88,16,5.0
11,6edf0be4b2267df1fa,15,6.0
13,7cfe354d9a64bf8173,15,7.0
19,cbc4bd40cd1687754,15,8.0
21,e0e0defbb9ec47f6f7,15,9.0
15,8bba390b53976da0cd,14,10.0


## 2. Calculate each user's average session time. 
- A session is defined as the time difference between a page_load and page_exit. 
- For simplicity, assume a user has only 1 session per day and if there are multiple of the same events on that day, consider only the latest page_load and earliest page_exit, with an obvious restriction that load time event should happen before exit time event . Output the user_id and their average session time.

In [5]:
df = pd.read_excel(f"{os.getcwd()}/data/fb_data.xlsx")
page_load_df = df[df['action']=='page_load'][['user_id', 'timestamp']]
page_exit_df = df[df['action']=='page_exit'][['user_id', 'timestamp']]

merge_df = pd.merge(page_load_df, page_exit_df, 
                    how='left', on='user_id',
                    suffixes=("_load", "_exit")
                    ).dropna()
# get session
merge_df['session'] = merge_df['timestamp_exit'] - merge_df['timestamp_load']
# filter out negative sessionns
merge_df = merge_df[merge_df['session'].dt.total_seconds()>0]
# add load date
merge_df['date_load'] = pd.to_datetime(merge_df['timestamp_load']).dt.date

# the latest page_load and earliest page_exit
merge_df1 = merge_df.groupby(['user_id','date_load']).agg({'timestamp_load': 'max', 'timestamp_exit': 'min'}).reset_index()
merge_df1['session'] = merge_df1['timestamp_exit'] - merge_df1['timestamp_load']
sessionn_rlt = merge_df1.groupby('user_id')['session'].mean().reset_index().rename(columns={'session':'average session time'})
sessionn_rlt

,user_id,average session time
0,0,0 days 00:31:23.500000
1,1,0 days 00:00:35


## 3. Write a query that'll identify returning active users. 
- A returning active user is a user that has made a second purchase within 7 days of any other of their purchases. 
- Output a list of user_ids of these returning active users.

In [6]:
amazon_transactions = pd.read_excel(f"{os.getcwd()}/data/User Purchases.xlsx")
amazon_transactions.head()

,id,user_id,item,created_at,revenue
0,1,109,milk,2020-03-03,123
1,2,139,biscuit,2020-03-18,421
2,3,120,milk,2020-03-18,176
3,4,108,banana,2020-03-18,862
4,5,130,milk,2020-03-28,333


In [7]:
amazon_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          100 non-null    int64         
 1   user_id     100 non-null    int64         
 2   item        100 non-null    object        
 3   created_at  100 non-null    datetime64[ns]
 4   revenue     100 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 4.0+ KB


In [8]:
df = amazon_transactions.sort_values(by=['user_id', 'created_at'], ascending=[True, True])
df['prev_date'] = df.groupby('user_id')['created_at'].shift()
df['days'] = df['created_at'] - df['prev_date']
df.head()

,id,user_id,item,created_at,revenue,prev_date,days
94,95,100,bread,2020-03-07,410,NaT,NaT
73,74,100,banana,2020-03-13,175,2020-03-07,6 days
19,20,100,banana,2020-03-18,599,2020-03-13,5 days
28,29,100,milk,2020-03-29,410,2020-03-18,11 days
26,27,101,milk,2020-03-01,449,NaT,NaT


In [9]:
returning_active_user = df[df['days'].dt.days<7].sort_values(by='user_id')['user_id'].unique().tolist()
returning_active_user 

[100,
 103,
 105,
 109,
 110,
 111,
 112,
 114,
 117,
 120,
 122,
 128,
 129,
 130,
 131,
 133,
 141,
 143,
 150]

## 6. Find the Index with Equal Left and Right Sum

Given a list of integers, find the index at which the sum of the left half of the list is equal to the right half. If there is no index where this condition is satisfied return -1.

Note: the number that lies on the index is calculated to the left side of the list.

Example 1:

Input:

nums = [1, 7, 3, 5, 6]
Output:

equivalent_index(nums) -> 2

In [35]:
def equivalent_index(nums):
    left_sum = 0
    right_sum = sum(nums)
    for i in range(len(nums)):
        left_sum += nums[i]
        right_sum -= nums[i]
        if left_sum == right_sum:
            return i
        
    return -1

nums = [1, 7, 3, 5, 6]
result = equivalent_index(nums)
print(result)

2


## 7. Write a function complete_address to create a single dataframe with complete addresses in the format of street, city, state, zip code.

- You’re given two dataframes. 
- One contains information about addresses and 
- the other contains relationships between various cities and states:

In [11]:
addresses = {"address": ["4860 Sunset Boulevard, San Francisco, 94105", "3055 Paradise Lane, Salt Lake City, 84103", "682 Main Street, Detroit, 48204", "9001 Cascade Road, Kansas City, 64102", "5853 Leon Street, Tampa, 33605"]}
cities = {"city": ["Salt Lake City", "Kansas City", "Detroit", "Tampa", "San Francisco"], "state": ["Utah", "Missouri", "Michigan", "Florida", "California"]}

df_addresses = pd.DataFrame(addresses)
df_cities = pd.DataFrame(cities)

In [12]:
def complete_address(df_addresses, df_cities):
    df_addresses[['street', 'city', 'zipcode']] = df_addresses['address'].str.split(', ', expand=True)
    df_addresses = df_addresses.drop(['address'], axis=1)
    df_addresses = df_addresses.merge(df_cities, on="city")
    df_addresses['address'] = df_addresses[['street', 'city', 'state', 'zipcode']].apply(lambda x: ', '.join(x), axis=1)
    df_addresses = df_addresses.drop(['street', 'city', 'state', 'zipcode'], axis=1)
    return df_addresses

In [13]:
df = complete_address(df_addresses, df_cities)
df

,address
0,"4860 Sunset Boulevard, San Francisco, Californ..."
1,"3055 Paradise Lane, Salt Lake City, Utah, 84103"
2,"682 Main Street, Detroit, Michigan, 48204"
3,"9001 Cascade Road, Kansas City, Missouri, 64102"
4,"5853 Leon Street, Tampa, Florida, 33605"


## 8. Transform this dataframe into a dataframe that contains the user ids and only the first name of each user.
- You’re given a dataframe containing a list of user IDs and their full names (e.g. ‘James Emerson’). 

In [14]:
users_df = pd.DataFrame({
    'user_id': [1034, 9430, 7281, 5264, 8995],
    'name': ["James Emerson", "Fiona Woodward", "Alvin Gross", "Deborah Handler", "Leah Xue"]
})

def first_name_only(users_df):
    users_df['name'] = users_df['name'].str.split(' ').str[0]
    return users_df


users_df = first_name_only(users_df)
users_df

,user_id,name
0,1034,James
1,9430,Fiona
2,7281,Alvin
3,5264,Deborah
4,8995,Leah


## 9. Write a function to return a dataframe containing every transaction with a total value of over $100. Include the total value of the transaction as a new column in the dataframe.
- The transactions dataframe contains transaction ids, product ids, and the total amount of each product sold.
- The products dataframe contains product ids and prices.

In [15]:
transactions = {"transaction_id" : [1, 2, 3, 4, 5], "product_id" : [101, 102, 103, 104, 105], "amount" : [3, 5, 8, 3, 2]}
products = {"product_id" : [101, 102, 103, 104, 105], "price" : [20.00, 21.00, 15.00, 16.00, 52.00]}

df_transactions = pd.DataFrame(transactions)
df_products = pd.DataFrame(products)

df_transactions

,transaction_id,product_id,amount
0,1,101,3
1,2,102,5
2,3,103,8
3,4,104,3
4,5,105,2


In [16]:
def transactions_over_100(df_transactions: pd.DataFrame, df_products: pd.DataFrame):
    trans_df = pd.merge(df_transactions, df_products, on='product_id')
    trans_df['total_value'] = trans_df['price']*trans_df['amount']
    trans_df = trans_df[trans_df['total_value']>100].drop(columns='price')
    return trans_df

trans_df = transactions_over_100(df_transactions, df_products)
trans_df


,transaction_id,product_id,amount,total_value
1,2,102,5,105.0
2,3,103,8,120.0
4,5,105,2,104.0


## 10. Fill the missinng price with median price

In [17]:
cheeses = {"Name": [
"Bohemian Goat", 
"Central Coast Bleu", 
"Cowgirl Mozzarella", 
"Cypress Grove Cheddar", 
"Oakdale Colby"], 
"Price" : [15.00, None, 30.00, None, 45.00]}

df_cheeses = pd.DataFrame(cheeses)
df_cheeses

,Name,Price
0,Bohemian Goat,15.0
1,Central Coast Bleu,NaN
2,Cowgirl Mozzarella,30.0
3,Cypress Grove Cheddar,NaN
4,Oakdale Colby,45.0


In [18]:
def cheese_median(df):
    df['Price'] = df['Price'].fillna(df['Price'].median())
    return df

df_cheeses = cheese_median(df_cheeses)
df_cheeses

,Name,Price
0,Bohemian Goat,15.0
1,Central Coast Bleu,30.0
2,Cowgirl Mozzarella,30.0
3,Cypress Grove Cheddar,30.0
4,Oakdale Colby,45.0


## 11. Calculated the t-value for the mean of ‘var’ against a null hypothesis. 
Extra: get p value

In [19]:
df = pd.DataFrame({
    'var': [-34, 40, -89, 5, -26]
})

mu0 = 1
df

,var
0,-34
1,40
2,-89
3,5
4,-26


In [20]:
def t_score(mu0, df):
  return (df.mean()-mu0)/(df.std()/sqrt(len(df)))

t_value = t_score(mu0, df)
t_value 

var   -1.015614
dtype: float64

In [21]:
from scipy import stats

t_stat, p_value = stats.ttest_1samp(df['var'], mu0)
print(t_stat, p_value )

# # Degrees of freedom
# df = sample_size - 1

# # Calculating the p-value (two-tailed)
# p_value = 2 * t.sf(np.abs(t_value), df)

-1.0156136626148202 0.36724985089457757


## 12. Create a new dataset with summary level information on their purchases including the columns:

- customer_id
- gender
- most_recent_sale
- order_count: order_count should display the total number of purchases that the customer has made.
- most_recent_sale should display the date of the customer’s most recent purchase. 

In [22]:
customers = {
"customer_id" : [5156, 2982, 1011, 3854, 2982],
"gender" : ["m", "f", "m", "f", "f"], 
"date of sale" : ["2021-01-04", "2021-02-15", "2021-03-01", "2021-03-21", "2021-04-12"]
}

customer_df = pd.DataFrame(customers)
customer_df

,customer_id,gender,date of sale
0,5156,m,2021-01-04
1,2982,f,2021-02-15
2,1011,m,2021-03-01
3,3854,f,2021-03-21
4,2982,f,2021-04-12


In [23]:
def customer_analysis(df):
    df = df.groupby(['customer_id','gender']).agg({'customer_id':'count', 'date of sale':'max'}).rename(columns={'customer_id':'order_count', 'date of sale':'most_recent_sale'})

    df = df.reset_index()
    return df
customer_df = customer_analysis(customer_df)
customer_df

,customer_id,gender,order_count,most_recent_sale
0,1011,m,1,2021-03-01
1,2982,f,2,2021-04-12
2,3854,f,1,2021-03-21
3,5156,m,1,2021-01-04


## 13. Write a solution to find the customer_number for the customer who has placed the largest number of orders.

In [24]:
import pandas as pd
data = [[1, 1], [2, 2], [3, 3], [4, 3]]
orders = pd.DataFrame(data, columns=['order_number', 'customer_number']).astype({'order_number':'Int64', 'customer_number':'Int64'})
orders

,order_number,customer_number
0,1,1
1,2,2
2,3,3
3,4,3


In [25]:
orders['customer_number'].mode().to_frame()

,customer_number
0,3


## 14. Write a solution to find the IDs of the invalid tweets. The tweet is invalid if the number of characters used in the content of the tweet is strictly greater than 15.

In [26]:
data = [[1, 'Vote for Biden'], [2, 'Let us make America great again!']]
tweets = pd.DataFrame(data, columns=['tweet_id', 'content']).astype({'tweet_id':'Int64', 'content':'object'})


In [27]:
def invalid_tweets(tweets: pd.DataFrame) -> pd.DataFrame:
    return tweets[tweets['content'].str.len()>15]['tweet_id'].to_frame()

tweets1 = invalid_tweets(tweets)
tweets1

,tweet_id
1,2


## 15. Write a solution to find the daily active user count for a period of 30 days ending 2019-07-27 inclusively. A user was active on someday if they made at least one activity on that day.

In [28]:
data = [[1, 1, '2019-07-20', 'open_session'], [1, 1, '2019-07-20', 'scroll_down'], [1, 1, '2019-07-20', 'end_session'], [2, 4, '2019-07-20', 'open_session'], [2, 4, '2019-07-21', 'send_message'], [2, 4, '2019-07-21', 'end_session'], [3, 2, '2019-07-21', 'open_session'], [3, 2, '2019-07-21', 'send_message'], [3, 2, '2019-07-21', 'end_session'], [4, 3, '2019-06-25', 'open_session'], [4, 3, '2019-06-25', 'end_session']]
activity = pd.DataFrame(data, columns=['user_id', 'session_id', 'activity_date', 'activity_type']).astype({'user_id':'Int64', 'session_id':'Int64', 'activity_date':'datetime64[ns]', 'activity_type':'object'})
activity.head()

,user_id,session_id,activity_date,activity_type
0,1,1,2019-07-20,open_session
1,1,1,2019-07-20,scroll_down
2,1,1,2019-07-20,end_session
3,2,4,2019-07-20,open_session
4,2,4,2019-07-21,send_message


In [29]:
def user_activity(activity: pd.DataFrame) -> pd.DataFrame:
    activity = activity[activity['activity_date'].between('2019-06-27', '2019-07-27')]
    activity = activity.groupby('activity_date')['user_id'].nunique().reset_index()
    activity = activity.rename(columns={'activity_date':'day', 'user_id':'active_users'})
    return activity

activity_ = user_activity(activity)
activity_

,day,active_users
0,2019-07-20,2
1,2019-07-21,2


## 16. Write a solution to find the average selling price for each product. average_price should be rounded to 2 decimal places.

In [30]:
data = [[1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15], [2, '2019-02-21', '2019-03-31', 30]]
prices = pd.DataFrame(data, columns=['product_id', 'start_date', 'end_date', 'price']).astype({'product_id':'Int64', 'start_date':'datetime64[ns]', 'end_date':'datetime64[ns]', 'price':'Int64'})
data = [[1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30]]
units_sold = pd.DataFrame(data, columns=['product_id', 'purchase_date', 'units']).astype({'product_id':'Int64', 'purchase_date':'datetime64[ns]', 'units':'Int64'})
prices

,product_id,start_date,end_date,price
0,1,2019-02-17,2019-02-28,5
1,1,2019-03-01,2019-03-22,20
2,2,2019-02-01,2019-02-20,15
3,2,2019-02-21,2019-03-31,30


In [31]:
units_sold

,product_id,purchase_date,units
0,1,2019-02-25,100
1,1,2019-03-01,15
2,2,2019-02-10,200
3,2,2019-03-22,30


In [32]:
def average_selling_price(prices: pd.DataFrame, units_sold: pd.DataFrame) -> pd.DataFrame:
    df = pd.merge(prices, units_sold, how='inner', on='product_id')
    df = df[df['purchase_date'].between(df['start_date'], df['end_date'])]
    df['total_price'] = df['price'] * df['units']
    rlt_df = df.groupby('product_id', as_index=False).apply(lambda x: (x.total_price.sum()/x.units.sum()).round(2))
    return rlt_df.rename(columns = {None:'average_price'})
rlt_df = average_selling_price(prices, units_sold)
rlt_df 

,product_id,average_price
0,1,6.96
1,2,16.96


## 17. Given a dictionary with weights, write a function random_key that returns a key at random with a probability proportional to the weights.

In [33]:
import random
weights = {'A': 1, 'B': 2}

def random_key(weights):
    weighted_keys = [key for key in weights for _ in range(weights[key])]
    return random.choice(weighted_keys)

weights = {'A': 1, 'B': 2}
key = random_key(weights)
print(key)

B


## 18 Write a function to use list comprehension to categorize age field to diff groups.

In [34]:
def categorize_age_groups(dataframe, age_column):
    """
    Categorize each entry into age groups based on the age column.

    Args:
    dataframe (pd.DataFrame): DataFrame containing the age data.
    age_column (str): The column name in the DataFrame that contains age data.

    Returns:
    pd.DataFrame: A new DataFrame with an added 'Age_Group' column.
    """
    if age_column not in dataframe.columns:
        raise ValueError(f"Column '{age_column}' not found in the DataFrame")

    dataframe['Age_Group'] = [
        'Youth' if age < 30 else 'Adult' if age <= 45 else 'Senior'
        for age in dataframe[age_column]
    ]
    return dataframe

# Example usage:
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [25, 30, 35, 40, 45],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']
}
df = pd.DataFrame(data)
df = categorize_age_groups(df, 'Age')
df


,Name,Age,City,Age_Group
0,Alice,25,New York,Youth
1,Bob,30,Los Angeles,Adult
2,Charlie,35,Chicago,Adult
3,David,40,Houston,Adult
4,Eve,45,Phoenix,Adult
